# Profiling and optimisation with Python

Also see: http://paris-swc.github.io/python-testing-debugging-profiling/

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Measuring total run times

In [2]:
import kmeans

In [3]:
def test_kmeans_two_gaussians(points, plot=False):
    """
    Use the kmeans algorithms with two 2-d Gaussians with different means and variances.
    One of the Gaussians has twice as many points as the other.
    
    points scales the total number of points.
    
    If `plot` is True, plot the data points and clusters (don't do this for many data points!).
    """
    N1, N2 = int(2*points), int(1*points)
    data = np.vstack([np.random.multivariate_normal([-2, -2], [[1, 0],   [0, 1]], size=N1),
                      np.random.multivariate_normal([1, 1],   [[0.5, 0], [0, 2]], size=N2)])
    labels = np.concatenate([np.zeros(N1), np.ones(N2)])
    initial = np.array([[-1, 0], [1, 0]])
    correct = kmeans.evaluate(data, labels, initial, plot=plot)
    return correct

In [7]:
%time test_kmeans_two_gaussians(100000)

Wall time: 1.36 s


0.97920666666666667

In [8]:
%timeit x = np.zeros(100)

845 ns ± 15 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Measuring detailed run times

In [11]:
%prun -s cumulative test_kmeans_two_gaussians(100000)

In [12]:
%load_ext snakeviz

In [13]:
%snakeviz test_kmeans_two_gaussians(100000)

 
*** Profile stats marshalled to file 'C:\\Users\\BDUBS\\AppData\\Local\\Temp\\tmp_dhqhbf7'. 


In [15]:
%load_ext line_profiler


The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [17]:
%lprun -f kmeans.kmeans test_kmeans_two_gaussians(100000)

### Warning: don't get lost in micro-optimisations

In [21]:
def is_prime(x):
    found = False
    for y in range(2, x):
        if x % y == 0:
            found = True
    return not found

def test_primes(low, high, n):
    test_data = np.random.randint(low, high, size=n)
    for number in test_data:
        # we throw away the results, we just use it to see how long it takes
        is_prime(number)

In [22]:
%lprun -f is_prime test_primes(0,1000,1000)

## Parallelization with multiple processes

**"Embarassingly parallel"** Problem: a problem that can be trivially separated into independent tasks

Typical examples:
  * Running the same calculation/analysis on different data sets
  * Running the same non-deterministic simulation several times
  * Parameter explorations

In [ ]:
%%time
import multiprocessing
pool = multiprocessing.Pool(4)
# also look into joblib library that we don't show here


#correct = np.zeros(10)
#for trial in range(10):
#    correct[trial] = test_kmeans_two_gaussians(100000)
    
correct = pool.map(test_kmeans_two_gaussians, np.ones(10)*100000)

In [26]:
correct

[0.97919999999999996,
 0.97877333333333338,
 0.97835000000000005,
 0.97842333333333331,
 0.97869666666666666,
 0.9788,
 0.97919333333333336,
 0.97871333333333332,
 0.97896666666666665,
 0.97832333333333332]